# KHSIC approach for disentangling content and style

## Data Prep

In [1]:
import time

In [2]:
%%time

alpha = 0.95
alpha_sk = 0.5 # for creating skewed data used to learn R
eta = 0.99
batch_size = 128
ns = 1 #specify number of style features


import numpy as np
from sklearn.linear_model import LogisticRegression
import torch
from numpy import load
import sys, json
from itertools import product
from sklearn import preprocessing


# Function for binarizing labels
def binarize(y):    
    y = np.copy(y) > 5
    return y.astype(int)

# Function for creating spurious correlations on 
def create_spurious_corr(z, z_t, y_og, spu_corr= 0.1, binarize_label=True):
    y_bin = binarize(y_og)
    mod_labels = np.logical_xor(y_bin, np.random.binomial(1, spu_corr, size=len(y_bin)))
    
    modified_images = z_t[mod_labels]
    unmodified_images = z[~mod_labels]
    all_z = np.concatenate((modified_images, unmodified_images), axis=0)
    style_labels = np.concatenate((np.zeros(len(modified_images)), np.ones(len(unmodified_images))), axis=None)
    
    all_img_labels = None
    
    if binarize_label:
        modified_imgs_labels = y_bin[mod_labels]
        unmodified_imgs_labels = y_bin[~mod_labels]
        all_img_labels = np.concatenate((modified_imgs_labels, unmodified_imgs_labels), axis=None)
    else:
        modified_imgs_labels = y_og[mod_labels]
        unmodified_imgs_labels = y_og[~mod_labels]
        all_img_labels = np.concatenate((modified_imgs_labels, unmodified_imgs_labels), axis=None)    
        
    return all_z, all_img_labels, style_labels.astype(int)


# Load saved image features

z_train_og = load('./data/Z_train_og_mnist_mlp.npy')
z_train_t = load('./data/Z_train_green_mnist_mlp.npy')

z_test_og = load('./data/Z_test_og_mnist_mlp.npy')
z_test_t = load('./data/Z_test_green_mnist_mlp.npy')

y_train_og = load('./data/train_labels_mnist.npy')

y_test_og = load('./data/test_labels_mnist.npy')


# Create spurious correlations on train and test sets

z_train_sk, train_labels_sk, t_labels_sk = create_spurious_corr(z_train_og, z_train_t, y_train_og, 
                                         spu_corr= alpha_sk, binarize_label=False)

z_train, train_labels, _ = create_spurious_corr(z_train_og, z_train_t, y_train_og, 
                                         spu_corr= alpha, binarize_label=False)

z_test_indist, indist_test_labels, _ = create_spurious_corr(z_test_og, z_test_t, y_test_og, 
                                                         spu_corr= alpha, binarize_label=False)

z_test_ood, ood_test_labels, _ = create_spurious_corr(z_test_og, z_test_t, y_test_og, 
                                                         spu_corr= 1-alpha, binarize_label=False)



# concatenate original and colored features
z_train_og_t = np.concatenate((z_train_og, z_train_t), axis=0)
t_train_labels = np.concatenate((np.zeros(len(z_train_og)), np.ones(len(z_train_t))), axis=None) 
z_test_og_t = np.concatenate((z_test_og, z_test_t), axis=0)
t_test_labels = np.concatenate((np.zeros(len(z_test_og)), np.ones(len(z_test_t))), axis=None) 


# # concatenate features with sytle labels..style labels are in column 0
# t_labels_z_train_og_t = np.concatenate((t_train_labels.reshape(-1,1), z_train_og_t), axis=1)

# # shuffle data in t_labels_z_train_og_t
# np.random.shuffle(t_labels_z_train_og_t)

# shuffled_train_og_t = t_labels_z_train_og_t[:,1:]
# shuffled_t_train_labels = t_labels_z_train_og_t[:,:1]


CPU times: user 7.11 s, sys: 8.98 s, total: 16.1 s
Wall time: 28.3 s


## MNIST data class distribustions per domain

In [3]:
import pandas as pd

# class distribution in original and colored images - class distribution is skewed
style_and_img_labels_z_train_sk_df = pd.DataFrame(np.concatenate((t_labels_sk.reshape(-1,1),
                                                                  train_labels_sk.reshape(-1,1),z_train_sk), axis=1))

print("class distribution - column 1 is class labels - column 0 is domain/environment labels")
class_distribution_per_domain = style_and_img_labels_z_train_sk_df.groupby([1,0]).count().iloc[:,0:1]
display(class_distribution_per_domain)


# shuffle data in style_and_img_labels_z_train_sk_df
style_and_img_labels_z_train_sk = style_and_img_labels_z_train_sk_df.to_numpy()
np.random.shuffle(style_and_img_labels_z_train_sk)

shuffled_train_og_t = style_and_img_labels_z_train_sk[:,2:]
shuffled_t_train_labels = style_and_img_labels_z_train_sk[:,:1]




class distribution - column 1 is class labels - column 0 is domain/environment labels


2
1   0        
0.0 0.0  2949
    1.0  2974
1.0 0.0  3424
    1.0  3318
2.0 0.0  2987
    1.0  2971
3.0 0.0  2976
    1.0  3155
4.0 0.0  2933
    1.0  2909
5.0 0.0  2721
    1.0  2700
6.0 0.0  3009
    1.0  2909
7.0 0.0  3153
    1.0  3112
8.0 0.0  2955
    1.0  2896
9.0 0.0  3010
    1.0  2939

# Find rotation matrix R by optimization----using KHSIC loss

In [4]:
%%time

import torch
import mctorch.nn as mnn
import mctorch.optim as moptim
from hsic_calculator import HSIC, normalized_HSIC


# # Reduce the samples size
# shuffled_train_og_t = shuffled_train_og_t[:10000]
# shuffled_t_train_labels = shuffled_t_train_labels[:10000]

dtype = torch.FloatTensor
n = shuffled_train_og_t.shape[0]
d = shuffled_train_og_t.shape[1]
k = int(shuffled_train_og_t.shape[1]*eta) # % of original number of features


# Initialize R
R = mnn.Parameter(manifold=mnn.Stiefel(d,k)).float()

# print("Initial R")
# display(R)

# Define Objective function 
def obj(z, e, W, n_s=1):
    z = torch.from_numpy(z).float()
    e = torch.from_numpy(e).float()
    MI_content_style = normalized_HSIC(torch.matmul(z, W[:,:n_s]), torch.matmul(z, W[:,n_s:]))
    MI_conten_env = normalized_HSIC(torch.matmul(z,W[:,n_s:]), e)
    MI_style_env = normalized_HSIC(torch.matmul(z,W[:,:n_s]), e)
    loss = (MI_content_style + MI_conten_env) - MI_style_env
    return loss

# Optimize - passing data in mini-batches
optimizer = moptim.rAdagrad(params = [R], lr=1e-2)

best_loss = 1e5
checkpoint = {}
R_mat = None
for epoch in range(100):
    for index in range(0, len(shuffled_train_og_t), batch_size):
        train_data_subset = shuffled_train_og_t[index:index+batch_size]
        style_labels_subset = shuffled_t_train_labels[index:index+batch_size]
        loss = obj(train_data_subset, style_labels_subset, R, ns)        
        # saving R with the smallest loss value so far
        if loss < best_loss:
            best_loss = loss
            print("Saving R, at epoch ", epoch)
            R_mat = R
            checkpoint = {'epoch': epoch, 'loss': loss, 'R': R}
            torch.save(checkpoint, 'checkpoint') 
            print("loss: ", loss)            
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

checkpoint

# print("R after optimization")
# display(R)
# (R.T)@R

/software/anaconda3/envs/invariance_env/lib/python3.10/site-packages/mctorch/nn/manifolds/stiefel.py:50: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1980.)
  q, r = torch.qr(X)


Saving R, at epoch  0
loss:  tensor(0.8831, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(0.4467, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(0.1956, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(0.0876, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(0.0788, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(0.0517, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.0776, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.1814, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.2048, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.4129, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.4352, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.4492, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.5703, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.6380, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.7

{'epoch': 75,
 'loss': tensor(-0.9425, grad_fn=<SubBackward0>),
 'R': Parameter containing:
 tensor([[-0.0238, -0.1703, -0.1091,  ...,  0.1422, -0.0260, -0.2008],
         [ 0.0394, -0.1584, -0.0736,  ..., -0.0174, -0.2711, -0.2770],
         [-0.1100, -0.1344,  0.0608,  ...,  0.0752,  0.0287,  0.0544],
         ...,
         [ 0.0111,  0.2455,  0.1454,  ..., -0.0420, -0.0895, -0.0916],
         [-0.0928,  0.0159,  0.0072,  ..., -0.0339, -0.0530,  0.1537],
         [-0.0343,  0.0400, -0.2661,  ...,  0.1609, -0.1369, -0.1777]],
        requires_grad=True)}

## Use the obtained rotation matrix R to disentangle content and style for OOD generalization

In [5]:
%%time

# Load saved R
# R_mat = torch.load('checkpoint')['R']
# R_mat = saved_R

# Obtain post-processed features
f_train_og = z_train_og @ R_mat.detach().numpy()  
f_train = z_train @ R_mat.detach().numpy()
f_test_indist = z_test_indist @ R_mat.detach().numpy()
f_test_ood = z_test_ood @ R_mat.detach().numpy()
f_test_og = z_test_og @ R_mat.detach().numpy()
f_test_t = z_test_t @ R_mat.detach().numpy()
f_test_og_t = z_test_og_t @ R_mat.detach().numpy()



########### Baseline results ###############
# Prediction Accuracies on image features extracted using a baseline model (mlp)
logistic_regression_on_baseline = LogisticRegression(multi_class='multinomial', solver='lbfgs', 
                                              random_state=0).fit(z_train,train_labels)                                                                                     
baseline_accuracy0 = logistic_regression_on_baseline.score(z_train, train_labels)
baseline_accuracy1 = logistic_regression_on_baseline.score(z_test_indist, indist_test_labels)
baseline_accuracy2 = logistic_regression_on_baseline.score(z_test_ood, ood_test_labels)   

# Trained on original baseline features, tested on colored features - no spurious correlations here
logistic_regression_on_baseline_og = LogisticRegression(multi_class='multinomial', solver='lbfgs',
                                                 random_state=0).fit(z_train_og,y_train_og)                                                                                     
baseline_og_accuracy0 = logistic_regression_on_baseline_og.score(z_train_og, y_train_og)
baseline_og_accuracy1 = logistic_regression_on_baseline_og.score(z_test_og, y_test_og)
baseline_transf_accuracy2 = logistic_regression_on_baseline_og.score(z_test_t, y_test_og)
####################################

# Classification task using all post-processed features except style features   
lr_model_new_HSIC_sp = LogisticRegression(multi_class='multinomial', solver='lbfgs',
                                    random_state=0).fit(f_train[:,1:],train_labels)
new_HSIC_sp_accuracy0 = lr_model_new_HSIC_sp.score(f_train[:,1:], train_labels)
new_HSIC_sp_accuracy1 = lr_model_new_HSIC_sp.score(f_test_indist[:,1:], indist_test_labels)
new_HSIC_sp_accuracy2 = lr_model_new_HSIC_sp.score(f_test_ood[:,1:], ood_test_labels)

# trained on original post-processed features, tested on transformed post-processed 
# features without style features   
lr_model_new_HSIC_no_sp = LogisticRegression(multi_class='multinomial', solver='lbfgs', 
                                    random_state=0).fit(f_train_og[:,1:],y_train_og)
new_HSIC_no_sp_accuracy0 = lr_model_new_HSIC_no_sp.score(f_train_og[:,1:], y_train_og)
new_HSIC_no_sp_accuracy1 = lr_model_new_HSIC_no_sp.score(f_test_og[:,1:], y_test_og)
new_HSIC_no_sp_accuracy2 = lr_model_new_HSIC_no_sp.score(f_test_t[:,1:], y_test_og)

# put all the results in a dictionary
results_log = {}

results_log['Baseline indist accuracy - spurious corr: '] = baseline_accuracy1
results_log['HSIC Approach indist accuracy - spurious corr: '] = new_HSIC_sp_accuracy1

results_log['Baseline ood accuracy - spurious corr: '] = baseline_accuracy2 
results_log['HSIC Approach ood accuracy- spurious corr: '] = new_HSIC_sp_accuracy2    

results_log['Baseline indist accuracy - no spurious corr: '] = baseline_og_accuracy1
results_log['HSIC Approach indist accuracy - no spurious corr: '] = new_HSIC_no_sp_accuracy1

results_log['Baseline ood accuracy - no spurious corr: '] = baseline_transf_accuracy2            
results_log['HSIC Approach ood accuracy - no spurious corr: '] = new_HSIC_no_sp_accuracy2   


results_log
 


/software/anaconda3/envs/invariance_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/software/anaconda3/envs/invariance_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/mod

CPU times: user 8min 25s, sys: 12min 13s, total: 20min 39s
Wall time: 19.4 s


/software/anaconda3/envs/invariance_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'Baseline indist accuracy - spurious corr: ': 0.9157,
 'HSIC Approach indist accuracy - spurious corr: ': 0.9124,
 'Baseline ood accuracy - spurious corr: ': 0.6661,
 'HSIC Approach ood accuracy- spurious corr: ': 0.6702,
 'Baseline indist accuracy - no spurious corr: ': 0.9121,
 'HSIC Approach indist accuracy - no spurious corr: ': 0.9118,
 'Baseline ood accuracy - no spurious corr: ': 0.2682,
 'HSIC Approach ood accuracy - no spurious corr: ': 0.3477}

<br>

<br>